In [1]:
# !rm -rf /Users/ryan/prosodic_data/data/english_wordtypes
import sys; sys.path.insert(0,'..')
from prosodic.imports import *

In [2]:
x='From fairest creatures we desire increase,'
line=Line(x)
wfm=line.wordform_matrix
wfl=wfm[0]
wfl

from FAI.rest CREA.tures we de.SIRE in.CREASE

In [3]:
@cache
def get_scansion_tree(nsylls, max_s=METER_MAX_S, max_w=METER_MAX_W):
    ll=get_possible_scansions(nsylls, max_s=max_s, max_w=max_w)
    ol=[]
    for scan_i,pos_l in enumerate(ll):
        for pos_i in range(1,len(pos_l)+1):
            mini_pos_l = pos_l[:pos_i]
            mini_numslots=sum(len(x) for x in mini_pos_l)
            if mini_numslots<4: continue
            # if mini_numslots not in dll: dll[mini_numslots]=[]
            # dll[mini_numslots].append([
            #     scan_i,
            #     pos_i+1,
            #     mini_numslots,
            #     pos_l,
            #     mini_pos_l,
            # ])
            ol.append(dict(
                scan_i=scan_i,
                num_meterpos=pos_i+1,
                num_meterslot=mini_numslots,
                parse_positions=tuple(pos_l),
                miniparse_positions=tuple(mini_pos_l),
            ))
            # break
    return pd.DataFrame(ol).sort_values('num_meterslot')

In [4]:
df=get_scansion_tree(10)
df

,scan_i,num_meterpos,num_meterslot,parse_positions,miniparse_positions
0,0,5,4,"(w, s, w, s, w, s, w, s, ww)","(w, s, w, s)"
886,168,3,4,"(ss, ww, s, w, ss, ww)","(ss, ww)"
289,55,4,4,"(ww, s, w, s, w, s, w, ss)","(ww, s, w)"
880,167,3,4,"(ss, ww, s, w, s, ww, s)","(ss, ww)"
295,56,4,4,"(ww, s, w, s, w, s, w, s, w)","(ww, s, w)"
...,...,...,...,...,...
519,97,9,10,"(s, w, s, w, ss, w, s, ww)","(s, w, s, w, ss, w, s, ww)"
514,96,9,10,"(s, w, s, w, s, ww, ss, w)","(s, w, s, w, s, ww, ss, w)"
509,95,10,10,"(s, w, s, w, s, ww, s, w, s)","(s, w, s, w, s, ww, s, w, s)"
498,93,10,10,"(s, w, s, w, s, w, ss, w, s)","(s, w, s, w, s, w, ss, w, s)"


In [11]:
bounded_scansions = set()
for g,gdf in list(df.groupby('num_meterslot')):
    logger.debug(f'Group: parses {g}')
    logger.debug(f'Number of parses in this group before filter: {len(gdf)}')
    gdf = gdf[~gdf.scan_i.isin(bounded_scansions)]
    logger.debug(f'Number of parses in this group after filter: {len(gdf)}')
    if not len(gdf): continue
    parses_str = gdf.miniparse_positions.unique()
    logger.debug(f'Number of unique mini-parses to try: {len(parses_str)}')
    parses = ParseList([Parse(wfl, parse_str) for parse_str in parses_str])
    parses.bound_parses(progress=False)
    resdf = pd.DataFrame([
        dict(
            miniparse_positions=tuple(parse.positions_ls),
            **parse.attrs
        ) 
        for parse in parses
    ])
    bothdf=resdf.merge(gdf, on='miniparse_positions')

    for i in bothdf.scan_i: bounded_scansions.add(i)

2023-10-08 14:53:15.783 | <module> | Group: parses 4 | 2976820725.py:3
2023-10-08 14:53:15.784 | <module> | Number of parses in this group before filter: 130 | 2976820725.py:4
2023-10-08 14:53:15.784 | <module> | Number of parses in this group after filter: 130 | 2976820725.py:6
2023-10-08 14:53:15.785 | <module> | Number of unique mini-parses to try: 10 | 2976820725.py:9
2023-10-08 14:53:15.788 | bound_parses | Bounding 10 with meter Meter(constraints=('w_peak', 'w_stress', 's_unstress', 'unres_across', 'unres_within'), categorical_constraints=(), max_s=2, max_w=2, resolve_optionality=True) | parsing.py:454
2023-10-08 14:53:15.790 | <module> | Group: parses 5 | 2976820725.py:3
2023-10-08 14:53:15.791 | <module> | Number of parses in this group before filter: 128 | 2976820725.py:4
2023-10-08 14:53:15.792 | <module> | Number of parses in this group after filter: 48 | 2976820725.py:6
2023-10-08 14:53:15.792 | <module> | Number of unique mini-parses to try: 6 | 2976820725.py:9
2023-10-08 

In [ ]:
df=get_scansion_tree(10)
df